# <center> Разбор кейса ML-инженера

## Обучим и протестируем модель

In [11]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
import pickle

In [12]:
ratings = pd.read_csv('data/ratings.csv')
books = pd.read_csv('data/books.csv')
tags = pd.read_csv('data/tags.csv')
book_tags = pd.read_csv('data/book_tags.csv')

In [13]:
books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [14]:
mapper = dict(zip(books.goodreads_book_id,books.book_id))

In [15]:
tags = pd.read_csv('data/tags_cleaned.csv')
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: mapper[x])

In [16]:
book_tags.head()

,goodreads_book_id,tag_id,count,id
1,1,11305,37174,27
4,1,33114,12716,27
5,1,11743,9954,27
6,1,14017,7169,27
10,1,27199,3857,27


In [17]:
ratings_coo = sparse.coo_matrix((ratings.rating,(ratings.user_id, ratings.book_id)))
feature_ratings  = sparse.coo_matrix(([1]*len(book_tags), (book_tags.id, book_tags.tag_id)))

Объявим вспомогательные константы для обучения модели:

In [18]:
#число потоков нашего процессора. Ставим 1, так как lightfm на macos ставится без OpenMP
NUM_THREADS = 1

#число параметров вектора 
NUM_COMPONENTS = 60

#число эпох обучения
NUM_EPOCHS = 10 

#зерно датчика случайных чисел
RANDOM_STATE = 42

На этапе создания модели мы используем библиотеку LightFM, чтобы сделать матричное разложение (ALS) наших рейтингов книг и получить два набора векторов. 

In [19]:
#Разбиваем наш датасет на обучающую и тестовую выборки
train, test = random_train_test_split(ratings_coo, test_percentage=0.2, random_state=RANDOM_STATE)

#Создаём модель
model = LightFM(
    learning_rate=0.05, #темп (скорость) обучения
    #loss='warp', #loss-функция
    no_components=NUM_COMPONENTS,#размерность вектора признаков
    random_state=RANDOM_STATE #генератор случайных чисел
)

#Обучаем модель
model = model.fit(
    train, #обучающая выборка
    epochs=NUM_EPOCHS, #количество эпох обучения
    num_threads=NUM_THREADS, #количество потоков процессора
    item_features=feature_ratings #признаки товаров (рейтинги книг)
)

Протестируем модель

In [20]:
#Тестируем нашу модель
precision_score = precision_at_k(
    model, #модель
    test, #тестовая выборка
    num_threads=NUM_THREADS, #количество потоков процессора
    k=10, #количество предложений
    item_features=feature_ratings #признаки товаров
).mean() #усредняем результаты
 
recall_score = recall_at_k(
    model, #модель
    test, #тестовая выборка
    num_threads=NUM_THREADS, #количество потоков процессора
    k=10, #количество предложений
    item_features=feature_ratings #признаки товаров
).mean() #усредняем результаты

print(recall_score, precision_score)

0.005063146124945426 0.01155891


Сохраним модель

In [21]:
with open('models/model.pkl', 'wb') as file:
    pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)

## Добавим эмбеддинги к модели и посмотрим, что получилось

In [22]:
with open('models/model.pkl', 'rb') as file:
    model = pickle.load(file)

In [23]:
# Достаём эбмеддинги
item_biases, item_embeddings = model.get_item_representations(features=feature_ratings)

print(item_biases.shape, item_embeddings.shape)

(10001,) (10001, 60)


In [25]:
import nmslib

In [26]:
#Инициализируем наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Начинаем добавлять наши книги в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [27]:
#Вспомогательная функция для поиска по графу
def nearest_books_nms(book_id, index, n=10):
    nn = index.knnQuery(item_embeddings[book_id], k=n)
    return nn

Найдем id книги 1984

In [28]:
#Отфильтруем только те, где в названии встречается подстрока "1984"
books[books['title'].apply(lambda x: x.lower().find('1984')) >= 0]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
12,13,5470,5470,153313,995,451524934,9.780452e+12,"George Orwell, Erich Fromm, Celâl Üster",1949.0,Nineteen Eighty-Four,...,1956832,2053394,45518,41845,86425,324874,692021,908229,https://images.gr-assets.com/books/1348990566m...,https://images.gr-assets.com/books/1348990566s...
845,846,5472,5472,2966408,51,151010269,9.780151e+12,"George Orwell, Christopher Hitchens",1950.0,Animal Farm & 1984,...,116197,118761,1293,1212,3276,16511,40583,57179,https://images.gr-assets.com/books/1327959366m...,https://images.gr-assets.com/books/1327959366s...
9795,9796,201145,201145,2563528,25,64440508,9.780064e+12,"Else Holmelund Minarik, Maurice Sendak",1968.0,A Kiss for Little Bear,...,11063,11604,126,87,284,1898,3053,6282,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


Теперь найдем все похожие книги и посмотрим на них

In [29]:
#Вызываем функцию для поиска ближайших соседей
print(nearest_books_nms(846, nms_idx))

(array([ 846,   55,   14,  809, 8140,   13, 2931,  173, 4582, 6727]), array([0.        , 0.14027512, 0.1433829 , 0.15845656, 0.18157768,
       0.2001133 , 0.21860588, 0.22225785, 0.24015301, 0.24789995],
      dtype=float32))


In [30]:
#Выделяем идентификаторы рекомендованных книг
nbm = nearest_books_nms(846, nms_idx)[0]
nbm

array([ 846,   55,   14,  809, 8140,   13, 2931,  173, 4582, 6727])

In [31]:
#Посмотрим на авторов и названия рекомендованных книг
books[books.book_id.isin(nbm)][['authors', 'title']]

,authors,title
12,"George Orwell, Erich Fromm, Celâl Üster",1984
13,George Orwell,Animal Farm
54,Aldous Huxley,Brave New World
172,Anthony Burgess,A Clockwork Orange
808,"Aldous Huxley, Christopher Hitchens",Brave New World / Brave New World Revisited
845,"George Orwell, Christopher Hitchens",Animal Farm / 1984
2930,"Edwin A. Abbott, Banesh Hoffmann",Flatland: A Romance of Many Dimensions
4581,"Thomas Hardy, Alexander Theroux",The Return of the Native
6726,George Orwell,Burmese Days
8139,Aldous Huxley,Brave New World Revisited


Сохраним эмбеддинги

In [32]:
with open('item_embeddings.pkl', 'wb') as file:
    pickle.dump(item_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)

### <center>ЗАЧЕМ НУЖЕН ПРОТОТИП?

Представим ситуацию: вы собрали сложную модель и хотите показать результаты её работы заказчику, чтобы он мог принять решение о её внедрении в продакшен. Вы приходите на совещание, демонстрируете код на ноутбуке, даже даёте модели какие-то данные и… ничего не происходит.

Чаще всего ставить задачи команде дата-сайентистов будут специалисты из других отраслей (продуктовая команда, маркетинг и так далее). Чтобы суметь донести смысл своей работы, получить качественную обратную связь и отследить возможные ошибки перед внедрением модели в продакшен, мы и строим прототипы.

Веб-приложения — один из самых удобных способов представить результаты работы в сфере науки о данных, именно поэтому такой формат является наиболее популярным для реализации прототипа. Однако многих специалистов в Data Science, не имеющих опыта веб-разработки, отпугивает идея создания подобных приложений, так как для реализации необходимы некоторые компетенции.

К счастью для нас, на сегодняшний день создавать наглядные и удобные для использования прототипы можно и без знания HTML, JavaScript, CSS и даже Flask, хотя последнее, безусловно, пригодится вам на практике.

**Например**, создавать прототипы можно с помощью фреймворка **Streamlit** при условии, что вы владеете Python. Streamlit выполняет самую сложную работу по созданию и компоновке веб-элементов, позволяя нам заниматься только данными.

### <center>ЧТО ТАКОЕ STREAMLIT?

**Streamlit** — это платформа приложений с открытым исходным кодом, созданная специально для DS-проектов. Эта платформа предоставляет Python-фреймворк и позволяет быстро строить интерактивный веб-интерфейс, пользуясь готовыми блоками (почти как Tilda для разработки сайтов). Большой плюс этого интерфейса в том, что впоследствии его можно загрузить на удалённый сервер, то есть выполнить деплой приложения.

**Примечание.** По умолчанию веб-сервер streamlit запускается через localhost на порте 8502(1?).

На данной странице представлены:

* Примеры использования Streamlit — небольшие скрипты, которые показывают возможности библиотеки, такие как анимация, визуализация графиков и интерактивных карт мира, работа с таблицами и прочее. Каждый пример сопровождён необходимым для его реализации кодом
* Официальные источники — ссылки на официальный сайт Streamlit, документацию по библиотеке и форум с обсуждением.
* Комплексные примеры проектов — ссылки на репозитории интересных проектов по машинному обучению, интерфейс которых реализован на Streamlit.

### <center>СОЗДАНИЕ ОСНОВЫ ДЛЯ ПРОТОТИПА

В этом разделе мы реализуем все необходимые для работы приложения вспомогательные функции и глобальные переменные.

**Важно.** Веб-приложения на Python обычно создаются в файлах формата .py, а не в Jupyter-ноутбуках (.ipynb), так как последние не приспособлены для удобной работы со скриптами запуска серверов.

Именно поэтому весь следующий код мы будем писать уже не в ноутбуке, а в файле app.py.

### <center>СОЗДАНИЕ ВИЗУАЛЬНОЙ ЧАСТИ ПРОТОТИПА

На примере интерфейса нашей рекомендательной системы книг мы рассмотрим основы создания интерфейса веб-приложения, построенного с помощью Streamlit.

Особенностью Streamlit является автоматическое непрерывное обновление интерфейса. Это означает при любом изменении в коде скрипт будет выполняться заново. То есть нам достаточно запустить веб-приложение один раз, а далее мы сможем вносить изменения в его интерфейс «на лету» и отсматривать эти изменения, просто перезагрузив страницу веб-браузера.

Давайте запустим текущую версию приложения — выполним в терминале следующую команду:
* $ streamlit run app.py

В результате выполнения команды в вашем браузере должна открыться новая вкладка:

Пока страница веб-приложения пустая — мы будем её заполнять. Однако предварительно необходимо определиться с общей концепцией будущего интерфейса. Воспользуемся методикой описания «пользователь — система»:

1. Пользователь входит в приложение и видит:
    * a название проекта;
    * b краткое описание проекта и краткую инструкцию по работе с приложением.
2. Пользователь в специальном окне вводит примерное название книги, которая интересует его, например несколько слов из её названия.
3. Система отображает список книг, подходящих под это название (выполняется частичный поиск).
4. Пользователь выбирает из этого списка ту книгу, которая ему нужна.
5. Система предлагает пользователю указать количество необходимых ему рекомендаций.
6. Пользователь указывает количество желаемых рекомендаций — .
7. Система составляет ранжированные рекомендации из  книг, наиболее похожих на ту, которую указал пользователь, и представляет их в удобном формате, например в виде таблицы.

Теперь, когда мы обсудили концепцию прототипа, перейдём к реализации интерфейса, попутно знакомясь с инструментарием библиотеки Streamlit.

#### **Заголовки**

Начнём с организационных компонентов приложения, в частности, с заголовков.

Добавим заголовок нашего проекта. Для этого предназначена функция st.title(), аргументом которой является текст заголовка.

**st.title("Recommendation System Of Books")**

**Примечание.** Библиотека Streamlit предлагает несколько способов размещения заголовков на веб-странице: например, с помощью функций st.header() и st.subheader() можно добавить дополнительные заголовки на двух разных уровнях. Также возможен вариант с применением синтаксиса Markdown для размещения таких заголовков. Для этого используется функция st.markdown(). Например:

* st.markdown("# Just like a header") — заголовок первого уровня;
* st.markdown("## Just like a subheader") — заголовок второго уровня.

#### **Текст**

Например, мы хотим отобразить в прототипе небольшое описание нашего приложения и инструкцию по работе с ним (текст будем приводить на английском языке, чтобы расширить потенциальную аудиторию).

Streamlit предоставляет множество способов отображения текста:

* st.text() — функция для отображения простого текста;
* st.markdown() — функция для отображения текста в формате Markdown;
* st.latex() — функция для отображения текста в формате LaTeX;
* st.code() — функция для отображения кода с возможностью указания языка программирования.

Мы предлагаем воспользоваться функцией st.markdown(). В аргумент этой функции передадим многострочный текст-описание нашего приложения:

#### **Текстовые виджеты**

Для ввода текстовых данных применяются виджеты st.text_input() и st.text_area(). Их основное различие в том, что первый лучше подходит для короткой текстовой строки, а второй — для большого объёма текстовых данных.

Давайте добавим в наш прототип поле для текстового ввода, где пользователь может указать приблизительное название интересующей его книги. Для этого воспользуемся функцией st.text_input(). В аргументы функции необходимо передать параметр label — приветственное сообщение для ввода.

Результат ввода заранее приведём к нижнему регистру для дальнейшего удобства поиска:

#### **Виджеты со множественным выбором**

На основе введённого пользователем приблизительного названия книги мы будем производить поиск неполных совпадений среди нашей базы книг. Полученные в результате фильтрации названия будут являться элементами некоторого выпадающего списка.

Есть несколько вариантов создания виджетов со множественным выбором для сбора ответов пользователей:

* «переключатель» st.radio()
* st.selectbox() предоставляет выпадающий список с возможностью выбора из него. Взаимодействие происходит следующим образом
* st.multiselect() — для случаев, когда пользователю необходимо ввести несколько выбранных вариантов

Для нашего прототипа подходит виджет st.selectbox(). Ему необходимо передать два аргумента:

* label — приветственная строка;
* options — массив из элементов выпадающего списка.

Из выпадающего списка пользователь выбирает нужную ему книгу. Если поле не пустое (результат работы функции st.selectbox() не является None), будет выводиться текст с указанием выбранной пользователем книги.

Проверьте работу виджетов: наберите в текстовом поле несколько ключевых слов из названий книг, например, "peace", "Anna", "1984" и т. д. В selectbox должны будут отображаться названия книг, удовлетворяющих вашему запросу.

#### **Числовые виджеты**

Далее пользователю предлагается указать количество необходимых ему рекомендаций. Здесь нам понадобятся числовые виджеты:

* st.number_input() предполагает неограниченный ввод путём увеличения счётчика
* st.slider() — ползунок, позволяющий пользователю устанавливать числовой ввод без набора каких-либо данных

Для простоты воспользуемся виджетом st.number_input(). В функцию необходимо передать два аргумента:

* label — приветственная строка;
* min_value — число, используемое по умолчанию.

Пусть по умолчанию ищется десять наиболее похожих книг. Результат, указанное пользователем число, запишем в переменную count_recomendation

Следующий шаг — найти указанное количество наиболее близких книг. Напомним, что для этих целей мы создали функцию nearest_books_nms(). Воспользуемся ей, указав:

* book_id — идентификатор книги, которую ввёл пользователь;
* index — индексы для реализации поиска;
* n — количество интересующих нас рекомендаций.

**Примечание.** Поскольку ближайшей к искомой книге всегда будет она сама, рекомендовать пользователю ту же книгу, скорее всего, неправильно, поэтому мы отбросим её. Тогда, чтобы порекомендовать пользователю n книг, мы должны указать в качестве параметра функции nearest_books_nms n+1 книг (count_recomendation+1):

#### **Таблицы**

Теперь у нас есть всё, чтобы отобразить пользователю рекомендации. Давайте представим их в виде таблицы.

Выше мы написали функцию get_recomendation_df(), которая помогает составлять DataFrame из рекомендаций, полученных с помощью kNN-поиска. Воспользуемся ей, указав:

* ids — идентификаторы рекомендованных книг;
* distances — меру расстояния от каждой рекомендации до заданной книги;
* name_mapper и author_mapper — словари для сопоставления id книги и её названия/автора.

После получения DataFrame мы сможем легко отобразить его в интерфейсе. В этом поможет функция st.dataframe(). В её аргументы необходимо передать лишь нужный нам DataFrame:

#### **Графики**

Streamlit нативно поддерживает множество вариаций графиков. Для отображения поддерживаемых элементов используется функция st.write(). Ниже представлено несколько примеров оформления функции для отображения графиков из официальной документации Streamlit:

* write(mpl_fig) — отображает график Matplotlib (включая Seaborn);
* write(altair) — отображает график Altair;
* write(graphviz) — отображает граф Graphviz;
* write(plotly_fig) — отображает график Plotly;
* write(bokeh_fig) — отображает график Bokeh.

Давайте с помощью библиотеки интерактивной визуализации Plotly построим столбчатую диаграмму, которая будет отображать значение косинусного расстояния для каждой рекомендованной книги. Столбчатую диаграмму будем строить с помощью функции px.bar() из plotly.expess

Получается, что Streamlit запустил сервер на нашей локальной машине и мы можем вводить название книг в рекомендательную систему и получать ответ.

Итак, мы создали визуальный прототип для обёртки нашей модели машинного обучения. Для этого мы использовали основные возможности молодого и очень простого в освоении фреймворка Streamlit. Как вы уже убедились, их довольно много: одни позволяют вводить числа и текст, другие — делать множественный выбор, третьи — отображать код, Markdown и графические элементы. Благодаря этому фреймворку мы можем полностью сосредоточиться на создании контента приложения, а не на реализации его интерфейса, что существенно уменьшает время создания прототипа.

Как видите, написать свою модель машинного обучения, а затем обернуть её в визуальный прототип несложно. В качестве дополнительной практики с библиотекой Streamlit предлагаем вам самостоятельно поработать с документацией и попробовать улучшить интерфейс приложения: добавить в него изображения, поработать с цветами, расположением кнопок и т. д.

Следующим шагом является развёртывание прототипа на сервере. Мы выведем приложение в общий доступ, и вашей рекомендательной системой смогут воспользоваться ваши друзья, коллеги и родственники.